## Working with Keras in DSX

In [ ]:
## Update Scikit-Learn
!pip install -U scikit-learn

## Check version of sklearn
!pip freeze | grep scikit-learn

## To install Theano
!pip install Theano

## To install TensorFlow
!pip install TensorFlow

## To install Keras
!pip install keras

## Version of Keras
!python -c "import keras; print(keras.__version__)"

## Develop Your First Neural Network With Keras

After loading the [Pima Indians data set](https://archive.ics.uci.edu/ml/datasets/pima+indians+diabetes) we define a simple multi-layer perceptron neural network and test its accuracy.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

## Set random seed for reproducibility
numpy.random.seed(206)

In [ ]:
# Access 'pima-indians-diabetes.csv' data file from the project.
dbtsDF = ProjectUtil.load_dataframe_from_file(pc, "pima-indians-diabetes.csv")
dbtsDF = dbtsDF.toPandas()

dbtsDF.head()

In [ ]:
## Split into input and output variables
X = dbtsDF.loc[:, '_c0':'_c7'].values
Y = dbtsDF.loc[:, '_c8'].values

In [ ]:
## Model definition
mlpMod = Sequential()
mlpMod.add(Dense(24, input_dim = 8, activation = "relu"))
mlpMod.add(Dense(12, activation = "relu"))
mlpMod.add(Dense(1, activation = "sigmoid"))

In [ ]:
## Compile Model
mlpMod.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
## Fit Model to Data
mlpMod.fit(X, Y, epochs = 150, batch_size = 20)

In [ ]:
## Evaluate Model
scores = mlpMod.evaluate(X, Y)
print("\n%s: %.2f%%" % (mlpMod.metrics_names[1], scores[1]*100))

_______________________

## Evaluating The Performance of Deep Learning Models in Keras

Next we'll explore various ways to evaluate the accuracy of your model.

#### Automatic Holdout/Validation Set

In [ ]:
## Automatic Verification Dataset in Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy

## Random seed 
numpy.random.seed(206)

## Re-use Pima Indians data, splitting into independent and dependent variables
X = dbtsDF.loc[:, '_c0':'_c7'].values
Y = dbtsDF.loc[:, '_c8'].values

## Create model
autoValMod = Sequential()
autoValMod.add(Dense(12, input_dim = 8, activation = "relu"))
autoValMod.add(Dense(8, activation = "relu"))
autoValMod.add(Dense(1, activation = "sigmoid"))

## Compile model
autoValMod.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

## Fit model on data
autoValMod.fit(X, Y, validation_split = 0.33, epochs = 150, batch_size = 10)

#### Manual Holdout/Validation Set

In [ ]:
## Manual Validation Set
import sklearn
from sklearn.model_selection import train_test_split

## Set seed again
seed = 7
numpy.random.seed(seed)

## Split into 67% for training data and 33% for testing
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state = seed)

## Define model hyperparameters
manValMod = Sequential()
manValMod.add(Dense(12, input_dim = 8, activation = "relu"))
manValMod.add(Dense(8, activation = "relu"))
manValMod.add(Dense(1, activation = "sigmoid"))

## Compile Model
manValMod.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])

## Fit Model
manValMod.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 150, batch_size = 10)

#### Manual _k_-Fold Cross-Validation

In [ ]:
## MLP for Pima Indians dataset with 10-fold cross validation
## Note: StratifiedKFold attempts to balance the number of instances of each class in the folds
from sklearn.model_selection import StratifiedKFold

## Define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)

cvscores = []

for train, test in kfold.split(X, Y):
    ## Create model
    kfMod = Sequential()
    kfMod.add(Dense(12, input_dim = 8, activation = "relu"))
    kfMod.add(Dense(8, activation = "relu"))
    kfMod.add(Dense(1, activation = "sigmoid"))
    
    ## Compile model
    kfMod.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])
    
    ## Fit model
    kfMod.fit(X[train], Y[train], epochs = 150, batch_size = 10, verbose = 0)
    
    ## Evaluate
    scores = kfMod.evaluate(X[test], Y[test], verbose = 0)
    
    print("%s: %.2f%%" % (kfMod.metrics_names[1], scores[1]*100))
    
    cvscores.append(scores[1] * 100)
    
    print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
    

## Using Keras Models with Scikit-Learn Functions

But why?  Well, you can take advantage of k-fold cross validation (as seen above) and functions for hyperparameter optimization.

#### _k_-fold Cross Validation

In [ ]:
## MLP for Pima Indians dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy

## Define our function to create our model, which we'll then pass to KerasClassifier
def createModel():
    ## Define model structure
    model = Sequential()
    model.add(Dense(12, input_dim = 8, activation = "relu"))
    model.add(Dense(8, activation = "relu"))
    model.add(Dense(1, activation = "sigmoid"))
    ## Compile
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return model

## Set seed
seed = 7
numpy.random.seed(seed)

## Use the KerasClassifier wrapper and pass the createModel() function as the build function
model = KerasClassifier(build_fn = createModel, epochs = 150, batch_size = 10, verbose = 0)

## Evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
results = cross_val_score(model, X, Y, cv = kfold)
print(results.mean())

That's a simpler way to do cross validation - we don't have to manually calculate the average score.  

#### Hyperparameter Optimization using Grid Search

Another advantage is to use grid search to find the optimimal hyperparameters.  

In [ ]:
## This will take a couple minutes, be patient ##

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

## Define new create_model() function with optimizer and init variables and their defaults
def create_model(optimizer = "rmsprop" , init = "glorot_uniform"):
  ## create model
  model = Sequential()
  model.add(Dense(12, input_dim = 8 , kernel_initializer = init , activation="relu"))
  model.add(Dense(8, kernel_initializer = init, activation ="relu"))
  model.add(Dense(1, kernel_initializer = init, activation = "sigmoid"))
  ## Compile model
  model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
  return model

gridSearchMod = KerasClassifier(build_fn = create_model, verbose = 0)

## Define grid search epochs, batch size and optimizer
optimizers = ["rmsprop" , "adam"]
inits = ["glorot_uniform", "normal", "uniform"]
epochs = [50, 100, 150]
batches = [5, 10, 20]

## Put these parameters into a python dictionary
param_grid = dict(optimizer = optimizers, epochs = epochs, batch_size = batches, init = inits)

## Pass the model and grid search parameters to the GridSearchCV class in scikit-learn, then fit it on the data
grid = GridSearchCV(estimator = gridSearchMod, param_grid = param_grid)
grid_result = grid.fit(X, Y)

## Summarize results ##
## Best score/hyperparameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## Accuracy statistics
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

The goal of these exercises was to show how wrapping Keras models can give you access to functions in `sklearn` that make your life easier as as data scientist.